In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf

In [18]:
data = pd.read_csv('../data/all_haiku.csv')

data = data.iloc[0:1000]

data['haiku'] = data['0']+' '+data['1']+' '+data['2']

data['haiku'] = data['haiku'].apply(lambda x: str(x).strip().replace('-','').lower())


In [93]:
#Leer los datos

text = ''

for i in data['haiku']:
    text += i + ' '

#vocab = sorted(set(text))
vocab = sorted(set(list(filter(lambda x: x != "", text.split(" ")))))

print(f'Text is {len(text)} long')
print(f'Text has {len(vocab)} unique characters')


Text is 50931 long
Text has 2728 unique characters


In [94]:
#Procesamiento del texto

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

txt_words = list(filter(lambda x: x != "", text.split(" ")))

text_as_int = np.array([char2idx[c] for c in txt_words])


In [95]:
seq_length = 100
examples_per_epoch = len(txt_words)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

fishing
boats
colors
of
the


In [96]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))


"fishingboatscolorsoftherainbowashwednesdaytryingtoremembermydreamsnowymornpouringanothercupofblackcoffeeshortestdayflamesdanceintheovenhazehalfthehorsehiddenbehindthehouselowsuntheladyinredonhighheelsadventthepassingstrangerfartstarnabubbleintheicesnowflakesnewasphaltintheholescrystalnight'gustsofrainoutsiderainthesoundofahorsegallopingthroughleaveswinterstarssuddenlyawhiffofperfumehungryhalfofthemoonhiddenrainanotherleafdownshariathesoundofonehandclapping"
"thesoundofgeesedrownedbythesoundofthetrainthismorningautumnsunmyshadowovertombstonesflyfishing;thesoundofthewindinthereeldecemberalongshadowjoinsanotherendofpathsnowflakesmeltingonthepondmorningfrostsheleavesfirsteveningwalksmelloftarbetweenpinesdachauablueskyabovethechimneysdeepautumn;theapplecolderinthetree.visitingthegravesstrongertheoctoberwindatmygrandparents'overthehedgeadragonflyeastabubbleburstsonsurfacefullmoonrainthewhitelilaclowmyhandon"
"herhipfullmoonthreepetalsfallfromthepurpleconeflower...almostsummerinstantmessagem

In [97]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))
    

Input data:  "fishingboatscolorsoftherainbowashwednesdaytryingtoremembermydreamsnowymornpouringanothercupofblackcoffeeshortestdayflamesdanceintheovenhazehalfthehorsehiddenbehindthehouselowsuntheladyinredonhighheelsadventthepassingstrangerfartstarnabubbleintheicesnowflakesnewasphaltintheholescrystalnight'gustsofrainoutsiderainthesoundofahorsegallopingthroughleaveswinterstarssuddenlyawhiffofperfumehungryhalfofthemoonhiddenrainanotherleafdownshariathesoundofonehand"
Target data:  "boatscolorsoftherainbowashwednesdaytryingtoremembermydreamsnowymornpouringanothercupofblackcoffeeshortestdayflamesdanceintheovenhazehalfthehorsehiddenbehindthehouselowsuntheladyinredonhighheelsadventthepassingstrangerfartstarnabubbleintheicesnowflakesnewasphaltintheholescrystalnight'gustsofrainoutsiderainthesoundofahorsegallopingthroughleaveswinterstarssuddenlyawhiffofperfumehungryhalfofthemoonhiddenrainanotherleafdownshariathesoundofonehandclapping"


In [98]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print(f'Step     {i}')
    print(f'   input: {int(input_idx), repr(idx2char[input_idx])}')
    print(f'   expected output: {int(target_idx), repr(idx2char[target_idx])}')

Step     0
   input: (891, "'fishing'")
   expected output: (257, "'boats'")
Step     1
   input: (257, "'boats'")
   expected output: (516, "'colors'")
Step     2
   input: (516, "'colors'")
   expected output: (1643, "'of'")
Step     3
   input: (1643, "'of'")
   expected output: (2405, "'the'")
Step     4
   input: (2405, "'the'")
   expected output: (1879, "'rainbow'")


In [99]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [100]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [101]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [102]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [103]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 2728) # (batch_size, sequence_length, vocab_size)


In [104]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           698368    
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 2728)          2796200   
Total params: 7,432,872
Trainable params: 7,432,872
Non-trainable params: 0
_________________________________________________________________


In [105]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()


In [106]:
sampled_indices

array([2487, 2577, 2081, 1700,  511, 2437, 2167, 2691, 1195, 1615, 1498,
       2068, 2100, 1012,  416,  476, 2521, 2446, 2375, 1898, 1677, 1145,
       1558, 1587, 1379, 2722, 1295, 2246, 1713,   18,  902, 2565,  894,
       1442,  120, 2246,  890, 1457, 1047, 2127, 1238,  268, 1883, 1950,
        971, 1059, 1898, 2519,  204, 2095, 2450, 1845, 1236, 1827, 1195,
       1448, 2383, 1984, 1226, 2157, 2500, 1964, 1265, 1773,  838, 1087,
        620,  983, 2349,  767, 2310, 1017,  905,  687,  119, 2712, 2703,
         11, 1969, 1108, 1055,  627,  205, 2514, 1402, 2478,  658, 1380,
        935, 2129,  608, 2726,  308,  150,  896, 2415,  393, 1787, 1859,
        846])

In [107]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))


Input: 
 'nightafluteplaysalongwiththerainlongexhaleasteadyrainonthetinroofsummerheatthepepperedscentofnasturtiumsafterrainamplifiedcricketsinthenightfreighttrainthoselittlestarssofarawayqueuetovotescentofsausagesinthecoldgreyskyfridayafternoonjustthebeesinlavenderspringgraffitiblossomalloverthewarehousewalllemonteaandlavenderoiltherainonrepeatopenwindowlemonscentedteatreeintherainsurfingwavesatinylizardoncorrugatedironmyownpacetinyflowerbloomsfromacrack'

Next Char Predictions: 
 "trashwalkedsharedpalecokethunderstorm—slowerworldhovernonamemiso!serviceshoesglidechalkclicktwotimetangleraysovenhibernatemovingnegativelogzillionkitespliffparties63flashingvoiceflakesmarkersasidesplifffishme?graveyardsilkeninkbornrainsrightfroggroom'sraystwinsberriesshimmertinyprisoninhibitionspotshovermarvelstearowsimprintslatstriproads'jamphotoshoppingfalteringhairsdamnedfurtherswayedgesstretchesgloveflavorditchashesyetyear...'romanhasgreynessdarkensberryturquoiselovingmakingtrail,deerlonefollowssiloscurv

In [108]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 2728)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       7.9112773


In [109]:
model.compile(optimizer='adam', loss=loss)


In [110]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [ ]:
#Training with 10 EPOCHS

EPOCHS=10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/10
1/1 [==============================] - 14s 14s/step - loss: 7.9113
Epoch 2/10
1/1 [==============================] - 6s 6s/step - loss: 7.9037
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 7.8928
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: 7.8697
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 7.7601
Epoch 6/10
1/1 [==============================] - 7s 7s/step - loss: 8.5524
Epoch 7/10


In [65]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/ckpt_10'

In [66]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


In [67]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            808192    
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 3157)           3235925   
Total params: 7,982,421
Trainable params: 7,982,421
Non-trainable params: 0
_________________________________________________________________


In [87]:
def generate_text(model, start_list):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_list]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (text_generated)

In [88]:
print(generate_text(model, start_list=['fishing', 'boats', 'colors', 'of', 'the', 'rainbow']))


['bows', 'the', 'mascara', 'buddha', 'to', 'strand', 'chickpea', 'face', 'morning', 'in', 'than', 'gloves', 'heard', 'looks', 'still', 'dandelions', 'city', 'of', 'of', 'always', 'bell', 'will', 'her', 'on', 'between', 'into', 'may', 'green', 'heat', 'me', 'rain', 'smell', 'off', 'covers', "i'm", 'sniffs', 'from', "o'", 'a', 'zen', 'moth', 'and', 'morning', 'rope', 'day', 'woman', 'bloomssudden', 'leaves', 'and', 'drums', 'through', 'full', 'day', "summer's", 'the', 'old', 'space', 'cracks', 'light', 'if', 'trying', 'around', 'for', 'to', 'end', 'hair', 'end', 'windshield', 'breeze', 'wool.estuary', 'feeling', 'and', 'in', 'and', 'endmoonlessness', 'faded', 'sun', 'time', 'sound', 'ricochets', 'my', 'child', 'wine', 'our', 'visiblerain', 'adjust', 'her', 'someone', 'bus', 'lilacs', 'late', 'finding', 'in', 'cloudsstart', 'sky', 'child', 'a', 'starsourdough', 'saxophoneshaving', 'directionnorth', 'my', '.', 'a', 'wipers', 'across', 'skies,', 'chill', 'a', '.', 'reading', 'empty', 'silen

In [ ]:
EPOCHS=10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])